In [1]:
import numpy as np
import benchmarkfunctions as bm
import utils
import tensorflow as tf
import gpfunctions as gp
import lstm_model


In [80]:
n_steps = 10
exp = np.arange(1,n_steps+1)[::-1]
print(exp)
pows = np.power(0.5,exp)
print(pows)
np.sum(pows)

[10  9  8  7  6  5  4  3  2  1]
[ 0.00097656  0.00195312  0.00390625  0.0078125   0.015625    0.03125
  0.0625      0.125       0.25        0.5       ]


0.9990234375

![](https://www.sfu.ca/~ssurjano/hart63.png)

In [215]:
x_6d = np.array([0.20169,0.150011,0.476874, 0.275332, 0.311652, 0.6573])
x_6d_scaled = x_6d*2 - 1
x_6d_scaled

array([-0.59662 , -0.699978, -0.046252, -0.449336, -0.376696,  0.3146  ])

In [216]:
def my_hartmann6(x, normalize=True):
    x = np.array(x)

    if normalize:
        x = (x+1)/2

    minv = -3.32237
    maxv = 38.7

    A = np.array([[10, 3, 17, 3.5, 1.7, 8],[0.05, 10, 17, 0.1, 8, 14], [3, 3.5, 1.7, 10, 17, 8],\
                  [17, 8, 0.05, 10, 0.1, 14]])
    P = 1e-4 * np.array([[1312, 1696, 5569, 124, 8283, 5886],[2329, 4135, 8307, 3736, 1004, 9991],\
                         [2348, 1451, 3522, 2883, 3047, 6650],[4047, 8828, 8732, 5743, 1091, 381]])
    alpha = np.array([1, 1.2, 3, 3.2])

    diff = np.power(np.subtract(P, x.T),2)
    ss   = np.sum(-A*diff, axis=1)
    print(ss.shape)
    exp  = np.exp(ss)

    y = -np.dot(alpha, exp)

    if normalize:
        y = 2*(y-minv)/(maxv-minv)-1

    return y
my_hartmann6(x_6d_scaled, normalize=True)

(4,)


-0.99999990535475936

-3.042457737830587

In [205]:
import imp
imp.reload(benchmarkfunctions)
imp.reload(utils)

<module 'utils' from '/Users/heytitle/projects/neural-network-project/utils.py'>

In [53]:
a = np.array([1,1]).reshape(1,2)
b = np.array([1,1]).reshape(1,2)
gp.rbf_kernel('np', a,b, 1 )

ValueError: 'axis' entry is out of bounds

In [81]:
np.power(0.5,np.arange(1,n_steps+1)[::-1])

array([ 0.00097656,  0.00195312,  0.00390625,  0.0078125 ,  0.015625  ,
        0.03125   ,  0.0625    ,  0.125     ,  0.25      ,  0.5       ])


\begin{align*}
E_{f, y_{1:T-1} 
\end{align*}


In [100]:
results = utils.combine_spearmint_results(utils.retrieve_spearmint_runs_from_dir('goldstein_price_2d', ['x1','x2']))


In [103]:
results.shape

(100, 21)

In [93]:
benchmarkfunctions.goldstein_price([-1,-1])

array([-0.99588822])

In [122]:
benchmarkfunctions.hartmann3(normed_x, normalize=True)

-0.9999999805635359

In [198]:
with tf.Session() as sess:
#     x = tf.constant([-1,-1,-1], shape=[1,4])
    y = benchmarkfunctions.hartmann3_tf(normed_x)

ValueError: Dimensions must be equal, but are 3 and 4 for 'sub_45' (op: 'Sub') with input shapes: [1,3,1], [4,3].

In [75]:
normed_x = (np.array([0.114614,0.555649,0.852547] )*2)-1

In [76]:
normed_x

array([-0.770772,  0.111298,  0.705094])

![](https://www.sfu.ca/~ssurjano/hart33.png)

<module 'utils' from '/Users/heytitle/projects/neural-network-project/utils.py'>

![](https://www.sfu.ca/~ssurjano/hart32.png)

In [153]:
def hartmann(x):
    A = np.array([[3.0, 10, 30],[0.1, 10, 35],[3.0, 10, 30],[0.1, 10, 35]])
    P = 0.0001 * np.array([[3689, 1170, 2673], [4699, 4387, 7470], [1091, 8732, 5547], [381, 5743, 8828]])
    alpha = np.array([1.0, 1.2, 3.0, 3.2])

    diff = np.power(np.subtract(P, x.T),2)
    print(x.T.shape)
    print(diff.shape)
    ss = np.sum(-A*diff, axis=1)
    print(ss.shape)
    exp = np.exp(ss)
    
    y = np.dot(alpha, exp)
    
    return -y
hartmann(np.array([0.114614,0.555649,0.852547]))

(3,)
(4, 3)
(4,)


-3.8627797869493365

In [52]:
def hartmann_for(x):
    external_sum = 0
    A = np.array([[3.0, 10, 30],[0.1, 10, 35],[3.0, 10, 30],[0.1, 10, 35]])
    P = 0.0001 * np.array([[3689, 1170, 2673], [4699, 4387, 7470], [1091, 8732, 5547], [381, 5743, 8828]])
    alpha = np.array([1.0, 1.2, 3.0, 3.2])
    for i in range(4):
        internal_sum = 0
        for j in range(3):
            internal_sum = internal_sum \
                        + A[i, j] * (x[j] \
                        - P[i, j]) ** 2
        print(internal_sum)
        external_sum = external_sum + alpha[i] * np.exp(-internal_sum)
    return -external_sum

In [104]:
hartmann_for(np.array([-1,-1,-1]))

66.28003033
127.734952501
111.29185353
148.964724461


-1.6403954926652363e-29

In [197]:
def my_hartmann3_tf(x):
    x = tf.reshape(x, [-1,1])

    x = (x+1)/2

    minv = -3.86278
    maxv = 18.06

    A = tf.constant([[3.0, 10, 30],[0.1, 10, 35],[3.0, 10, 30],[0.1, 10, 35]], dtype=tf.float64)
    P = 0.0001 * tf.constant([[3689, 1170, 2673], [4699, 4387, 7470], [1091, 8732, 5547], [381, 5743, 8828]], dtype=tf.float64)
    alpha = tf.constant([1.0, 1.2, 3.0, 3.2], dtype=tf.float64, shape= [1,4])

    diff = tf.subtract(P, tf.transpose(x))**2

    ss = tf.reduce_sum( tf.multiply(-A, diff ), 1)
    ss = tf.reshape(ss, [-1,1])
    exp  = tf.exp(ss)
    
    y = -tf.matmul(alpha, exp)

    y = 2*(y-minv)/(maxv-minv)-1

    return tf.reshape(y, (-1,1))

with tf.Session() as sess:
    y = my_hartmann3_tf(normed_x)
    y = sess.run(y)
    ttt = np.array(y).flatten()[0]
    print(ttt)

-0.999999980564


In [176]:
ttt

array([[-0.99999998]])

In [137]:
y = tf.constant(2) * 5
with tf.Session() as sess:
    print(sess.run(y*2))

20


In [221]:
run_logs = utils.retrieve_spearmint_runs_from_dir('hartmann_6d', ['x1','x2','x3','x4','x5','x6'], runs_dir='runs')

In [222]:
run_logs[0]

,job_id,job_run,y,x1,x2,x3,x4,x5,x6
0,1498414713,1,-0.842118,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,1498414713,2,-0.865926,0.0,0.0,0.0,0.0,0.0,0.0


In [223]:
normed_x

array([-0.770772,  0.111298,  0.705094])

In [1]:
import skopt

In [4]:
skopt['gp_minimize']

TypeError: 'module' object is not subscriptable

In [48]:
import imp
imp.reload(bm)

<module 'benchmarkfunctions' from '/Users/heytitle/projects/neural-network-project/benchmarkfunctions.py'>

In [15]:
benchmarkfunctions.styblinski4( np.array([-2.903534]*4)/5.0, normalize=False)

-156.66466281508559

In [20]:
with tf.Session() as sess:
    x = tf.placeholder(tf.float32, [None, 4])
    y = benchmarkfunctions.styblinski4_tf( x, normalize=False)
    
    result = sess.run(y, feed_dict = { x :  np.array([-2.903534]*4).reshape(1,-1)/5.0 } )
    print(result)

[[-156.66467285]]


In [50]:
bm.styblinski4([-1]*4, True)

1.0

In [29]:
np.array([1,2]) * np.array([3,5])

array([ 3, 10])

In [41]:
with tf.Session().as_default():
    d=bm.parabolasin_tf(np.array((-0.29168361,-0.08170144), dtype=np.float32).reshape(1,2)).eval()
    print(d)

[[-0.71154112]]


In [42]:
[1]

[1]

# Validate GP

In [144]:
imp.reload(gp)

<module 'gpfunctions' from '/Users/heytitle/projects/neural-network-project/gpfunctions.py'>

In [4]:
dim = 2
kernel = 'rbf'
kernel_func = gp.kernel_function(kernel)
idx = 2
n_bumps = 6
l = 2/n_bumps*np.sqrt(dim)

X, A, minv, maxv = utils.loadData(dim, 'testing', kernel)


In [5]:
x0 = np.array([-1.0,-1.0], dtype=np.float64)

In [6]:
fun = lambda x, i, engine : np.asscalar(gp.benchmark_prior1(engine, X[np.newaxis,i], A[np.newaxis,i],\
                                minv[np.newaxis,i], maxv[np.newaxis,i], l, kernel_func, \
                                np.array(x)[np.newaxis,np.newaxis,:]))


In [16]:
fun(x0,idx, 'np')

0.15406050194459067

In [15]:
import imp
imp.reload(gp)

<module 'gpfunctions' from '/Users/heytitle/projects/neural-network-project/gpfunctions.py'>

In [17]:
def find_y_from_lstm(X_test, A_test, min_test, max_test, kernel, dim, loss, n_steps=20, debug=False, dataset="normal"):
    
    model = utils.get_trained_model(dim=dim, kernel=kernel, loss=loss)    
    starting_point = x0
    print(starting_point)

    data_Xt = X_test
    data_At  = A_test
    data_mint = min_test.reshape(-1,1)
    data_maxt = max_test.reshape(-1,1)

    model_params = lstm_model.load_model_params(model, debug=False)

    with tf.Session() as sess:
        params_dict = {
            'n_bumps': model_params['n_bumps'],
            'dim' : model_params['dim'],
            'n_hidden': model_params['n_hidden'],
            'forget_bias': model_params['forget_bias'],
            'n_steps': n_steps,
            'l': model_params['gp_length'],
            'scope': model_params['scope'],
            'function': gp.dataset_function(dataset),
            'kernel': gp.kernel_function(kernel)
        }
        
        Xt, At, mint, maxt, samples_x, samples_y, x_0, _, _ = lstm_model.build_training_graph(**params_dict)

        saver = tf.train.Saver()
        saver.restore(sess, model_params['model_path'])

        feed_dict = {
            Xt: data_Xt,
            At: data_At,
            mint: data_mint,
            maxt: data_maxt,
            x_0: np.array(starting_point).reshape(1,-1)
        }

        res, res_x = sess.run([samples_y, samples_x], feed_dict=feed_dict)

        return np.array(res).reshape(n_steps,-1).T
find_y_from_lstm(X,A, minv, maxv, kernel, dim, 'MIN', dataset='prior1')[idx,0]

[-1. -1.]
INFO:tensorflow:Restoring parameters from ./trained_models/rnn-cell-2d-1498485717/model


0.15405981

In [23]:
np.random.randint(n=10)

TypeError: randint() takes at least 1 positional argument (0 given)

In [25]:
np.random.randint(0,10000,(10))

array([6383, 4136, 7695, 3451, 2352, 1452, 1053, 6323, 3781, 1098])